## Analiz adımları

### Kalite kontrolü
```
[fastqc]
tool = ${PIPELINE:fastqc}
input_from = input_files
args =  input_placeholder
    -t 8
    --outdir=${PROJECT:output_dir}
dump_dir = fastqc
```

https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

komut:
    fastqc Sample_R1.fq.gz


### Hizalama/Haritalama (Alignment/Mapping) & Okumaların hazırlanması

https://www.biostars.org/p/212136/
    
```
[Fastq2Bam]
tool = ${PIPELINE:bwa}
sub_tool = mem
input_from = input_files
args = -t 8
    -R "@RG\tID:name_placeholder\tSM:name_placeholder\tPL:illumina\tLB:lib1\tPU:foo"
    ${GENERAL:resource_dir}/${GENERAL:reference_genome}
    input_placeholder |
    samtools collate -@8 -O - - |
    samtools fixmate -@8 -m -O bam - - |
    samtools sort -@8 -O bam - |
    samtools markdup -@8 -O bam - output_placeholder && samtools index -@8 output_placeholder
named = True
suffix = _bwa
ext = bam
input_multi = paired
```

#### BWA mem

http://bio-bwa.sourceforge.net/bwa.shtml

bwa referans_genome Sample_R1.fq.gz Sample_R2.fq.gz > Sample.sam

#### Samtools collate
komut:

    samtools collate Sample.sam -o Sample_collate.bam

amaç:
    
    Okumaları grupla.


#### Samtools fixmate

http://www.htslib.org/doc/samtools.html

komut:

    samtools fixmate Sample_collate.sam -o Sample_collate_fixmate.bam

amaç:
    
    Eş okumaları işaretle markdup için.


#### Samtools sort
komut:
    
    samtools sort Sample_collate_fixmate.bam -o Sample_collate_fixmate_sort.bam
amaç:
    
    Okumaları genomik pozisyonu göre sırala.

#### Samtools markdup
komut:
    
    samtools markdup Sample_collate_fixmate_sort.bam -o Sample_collate_fixmate_sort_markdup.bam
amaç:
    
    Tekrar okumalarını işaretle.

#### Samtools index
komut:
    
    samtools index Sample_collate_fixmate_sort_markdup.bam
amaç:
    
    bam dosyasını indexle.
    

### Varyant Çağırma
```
[deepvariant]
container = ${PIPELINE:docker_deepvariant}
tool = ${PIPELINE:deepvariant}
input_from = input_files
args =  --model_type=WES
	--ref=/resources/${GENERAL:reference_genome}
	--regions="/bed/${PROJECT:bed_file}"
	--reads=/io/input_placeholder
	--output_vcf=/io/output_placeholder
	--num_shards=8
	&& ${PIPELINE:docker_deepvariant} sh -c "grep '^#' /io/output_placeholder > /io/output_placeholder_ && grep -v '#' /io/output_placeholder | grep -v 'RefCall' >> /io/output_placeholder_ && mv -f /io/output_placeholder_ /io/output_placeholder"
ext = vcf
output_dir = .
input_dir = .
```

https://google.github.io/deepvariant/posts/2020-02-20-looking-through-deepvariants-eyes/

komut:

    deepvariant --model_type=WES --ref=referans_genome  --regions=bed_file --reads=Sample.bam --output_vcf=Sample.vcf

### Anotasyon
```
[runVEPpick]
container = ${PIPELINE:docker_vep}
tool = ./vep
input_from = input_files
args =  --cache --refseq --everything
	# --offline
	--assembly GRCh37
	--flag_pick --pick_order ccds,tsl,rank,biotype,mane,appris,canonical,length
	--fork 8
	--format vcf --vcf --force_overwrite
	--fasta=/resources/${GENERAL:reference_genome}
	--dir_cache /opt/vep/.vep/ --dir_plugins /opt/vep/.vep/Plugins/
	--canonical
	--input_file /io/input_placeholder
	--output_file /io/output_placeholder
	--compress_output bgzip
	--plugin GO
	--plugin Phenotypes
	# --plugin Draw,output_placeholder,2000,100
	-custom /resources/hg19/clinvar.vcf.gz,ClinVar,vcf,exact,0,CLNSIG,CLNHGVS
	&& ${PIPELINE:docker_vep} rm /io/input_placeholder
suffix = _vep
ext = vcf.gz
output_dir = .
input_dir = .
```

https://www.ensembl.org/info/docs/tools/vep/script/vep_options.html

komut:
    ./vep --cache -i Sample.vcf -o Sample.vcf

